In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from tqdm import tqdm
import sys
sys.path.extend(["..", "../../backbone","../.."])
from dataloader import create_dataloader
from metric import Metric
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve, roc_auc_score, f1_score
from tabulate import tabulate
torch.cuda.empty_cache()


# Set device to GPU if available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Current device: {torch.cuda.get_device_name(torch.cuda.current_device())}" if torch.cuda.is_available() else "Current device: CPU")

Current device: NVIDIA A100 80GB PCIe


In [2]:
# Define hyperparameters
batch_size = 16
num_epochs = 200
learning_rate = 0.000001
image_size = 384
num_workers = 4
num_labels = 21
in_channels = 3
num_classes = 21
# thresholds = [0.84, 0.21, 0.79, 0.11, 0.89, 0.9, 0.85, 0.05, 0.15, 0.81, 0.97, 0.06, 0.02, 0.89, 0.21, 0.95, 0.25, 0.23, 0.44, 0.04, 0.01]
# thresholds = [0.23, 0.28, 0.28, 0.35, 0.96, 0.29, 0.37, 0.58, 0.27, 0.42, 0.47, 0.51, 0.24, 0.35, 0.39, 0.36, 0.6, 0.83, 0.61, 0.09, 0.1]
thresholds = [0.18, 0.13, 0.36, 0.54, 0.23, 0.31, 0.38, 0.96, 0.26, 0.8, 0.69, 0.41, 0.29, 0.39, 0.42, 0.83, 0.62, 0.53, 0.29, 0.21, 0.15]
data_dir = '../../../../data/GT-main'
omit = [None]

In [3]:
# Create test dataloader
test_dataloader = create_dataloader(data_dir=data_dir, batch_size=batch_size, num_workers=num_workers, size=image_size, phase='test', omit=omit)

384
../../../../data/GT-main/./set1/test.csv


In [4]:
model = torch.load('../models/2/rs-2-rf.pth', map_location=torch.device('cpu'))
model.to(device)

ResNet152d_2(
  (features): ResNet(
    (conv1): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_st

In [5]:
def calculate_metrics(test_labels, test_preds, thresholds):
    num_classes = test_labels.shape[1]
    metrics_dict = {}
    thresholded_test_preds = np.where(test_preds > np.array(thresholds).reshape(1, -1), 1, 0)

    for label in range(num_classes):
        TP = np.sum((test_labels[:, label] == 1) & (thresholded_test_preds[:, label] == 1))
        FP = np.sum((test_labels[:, label] == 0) & (thresholded_test_preds[:, label] == 1))
        TN = np.sum((test_labels[:, label] == 0) & (thresholded_test_preds[:, label] == 0))
        FN = np.sum((test_labels[:, label] == 1) & (thresholded_test_preds[:, label] == 0))

        precision = TP / (TP + FP) if (TP + FP) > 0 else 1.0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 1.0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 1.0
        auc = roc_auc_score(test_labels[:, label], test_preds[:, label])

        metrics_dict[label] = {'Precision': precision, 'Recall': recall, 'F1': f1, 'AUC': auc}

    return metrics_dict

In [6]:
# Evaltuate the model on test set
model.eval()
test_preds, test_labels = [], []
with torch.no_grad():
    for images, labels in tqdm(test_dataloader):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        outputs = torch.sigmoid(outputs)

        test_preds += outputs.tolist()
        test_labels += labels.tolist()

test_labels = np.array(test_labels)
test_preds = np.array(test_preds)

  0%|          | 0/34 [00:01<?, ?it/s]


AttributeError: 'ResNet152d_2' object has no attribute 'relu2'

In [ ]:
metrics_dict = calculate_metrics(test_labels, test_preds, thresholds)

# Print the dictionary in tabular format using the tabulate library
headers = ['Label', 'Precision', 'Recall', 'F1', 'AUC']
table = []
for label in range(len(metrics_dict)):
    row = [label]
    for metric in ['Precision', 'Recall', 'F1', 'AUC']:
        value = metrics_dict[label][metric]
        row.append('{:.4f}'.format(value))
    table.append(row)

print(tabulate(table, headers=headers))

  Label    Precision    Recall      F1     AUC
-------  -----------  --------  ------  ------
      0       0.171     1       0.292   0.8705
      1       0.2188    1       0.359   0.8789
      2       0.0588    1       0.1111  0.5
      3       1         0       0       0.5
      4       0.0515    1       0.0979  0.5
      5       0.0717    1       0.1338  0.816
      6       0.0404    1       0.0777  0.7416
      7       1         0       0       0.5
      8       0.0368    1       0.0709  0.5
      9       1         0.2308  0.375   0.6486
     10       0.8       1       0.8889  0.9987
     11       0.0165    1       0.0325  0.8303
     12       0.0239    1       0.0467  0.5
     13       0.0239    1       0.0467  0.5
     14       0.0147    1       0.029   0.8706
     15       1         0       0       0.5
     16       0.75      0.6     0.6667  0.7981
     17       0         0       1       0.4944
     18       0.0239    1       0.0467  0.5
     19       0.1397    1       0.2452  0